In [1]:
from google.colab import drive 

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!ls "/content/drive/MyDrive/AIAssignment3"

Food101DataSet.gz  Untitled0.ipynb


In [2]:
import os
os.chdir('/content/drive/MyDrive/AIAssignment3')

In [3]:
import tensorflow_datasets as tfds

dataset, info = tfds.load('food101', with_info=True, as_supervised=True)

In [4]:
train_dataset = dataset['train']
val_dataset = dataset['validation']

In [5]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
IMG_SIZE = 224

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_dataset = train_dataset.map(preprocess)
val_dataset = val_dataset.map(preprocess)


In [6]:
BATCH_SIZE = 128

train_dataset = train_dataset.shuffle(buffer_size=1000)
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

val_dataset = val_dataset.batch(BATCH_SIZE)
val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)


In [7]:
base_model = ResNet50(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                      include_top=False,
                      weights='imagenet')

for layer in base_model.layers:
    layer.trainable = False

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

prediction_layer = tf.keras.layers.Dense(101, activation='softmax')

model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])


94765736/94765736 [==============================] - 1s 0us/step


In [8]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_dataset,
          epochs=1,
          validation_data=val_dataset)


592/592 [==============================] - 353s 548ms/step - loss: 4.5845 - accuracy: 0.0185 - val_loss: 4.5137 - val_accuracy: 0.0286


In [ ]:
#should work but dont have enough RAM
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
from sklearn import svm
from sklearn.metrics import accuracy_score
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from sklearn.decomposition import PCA

trainSVM_dataset = dataset['train']
valSVM_dataset = dataset['validation']
model = VGG16(weights='imagenet', include_top=False)

def extract_features(dataset):
    features = []
    labels = []
    for image, label in dataset:
        image = tf.image.resize(image, (224, 224))
        image = tf.reshape(image, (1, 224, 224, 3))
        feature = model.predict(image, batch_size=32)
        features.append(feature.flatten())
        labels.append(label.numpy())
    return np.array(features), np.array(labels)

  
train_features, train_labels = extract_features(trainSVM_dataset)
val_features, val_labels = extract_features(valSVM_dataset)


1/1 [==============================] - 0s 73ms/step


In [ ]:
clf = svm.SVC(kernel='linear')

clf.fit(train_features, train_labels)


In [ ]:
val_pred = clf.predict(val_features)

val_acc = accuracy_score(val_labels, val_pred)

print('Validation accuracy:', val_acc)


In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

In [12]:
val_loss, val_acc = model.evaluate(val_dataset)

val_pred = np.argmax(model.predict(val_dataset), axis=1)

val_labels = np.concatenate([y.numpy() for x, y in val_dataset], axis=0)

print('ResNet Validation Accuracy:', val_acc)
print('ResNet Validation Precision:', precision_score(val_labels, val_pred, average='weighted'))
print('ResNet Validation Recall:', recall_score(val_labels, val_pred, average='weighted'))
print('ResNet Validation F1 score:', f1_score(val_labels, val_pred, average='weighted'))


198/198 [==============================] - 75s 376ms/step
ResNet Validation Accuracy: 0.028633663430809975
ResNet Validation Precision: 0.042867951776041135
ResNet Validation Recall: 0.028633663366336635
ResNet Validation F1 score: 0.01168109988727608


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
val_features = np.array([x.numpy().flatten() for x, y in val_dataset])
val_labels = np.concatenate([y.numpy() for x, y in val_dataset], axis=0)

val_pred = model.predict(val_features)

print('SVM Validation Accuracy:', accuracy_score(val_labels, val_pred))
print('SVM Validation Precision:', precision_score(val_labels, val_pred, average='weighted'))
print('SVM Validation Recall:', recall_score(val_labels, val_pred, average='weighted'))
print('SVM Validation F1 score:', f1_score(val_labels, val_pred, average='weighted'))
